In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, auc, roc_curve, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [9]:
df = pd.read_csv('data/Clean_Dataset.csv')

In [10]:
df.head()

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [11]:
X = df.drop(columns=['price'
                    ,'flight'
                    ,'Unnamed: 0'
                    #,'MentalHealth'
                    #,'Race'
                    #,'GenHealth'
                    #,'Asthma'
                    #,'Stroke'
                    #,'Smoking'
                    #,'Sex'
                    #,'SleepTime'

#                    DiffWalking_Yes                        0.201258
#Stroke_Yes                             0.196835
#Diabetic_Yes                           0.183072
#GenHealth_Poor                         0.174662
#PhysicalHealth                         0.170721
#GenHealth_Fair                         0.147954
#KidneyDisease_Yes                      0.145197
#AgeCategory_80 or older                0.143041
#Smoking_Yes
                     ],axis=1)

In [12]:
X.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,SpiceJet,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1
1,SpiceJet,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1
2,AirAsia,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1
3,Vistara,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1
4,Vistara,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1


In [13]:
y = df['price']

In [225]:
# here we do not have categorial variables, so we do not need to do one-hot encoding
# y=df['HeartDisease'].replace(to_replace=['No', 'Yes'], value=[0, 1])

In [14]:
# Creation of Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
#, OneHotEncoder
#, StandardScaler#MinMaxScaler  
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
#StandardScaler() 
oh_transformer = OrdinalEncoder()
#OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("Numerical", numeric_transformer, num_features)
         #("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [15]:
X = preprocessor.fit_transform(X)

In [16]:
X.shape

(300153, 9)

In [17]:
X

array([[ 4.        ,  2.        ,  2.        , ...,  1.        ,
        -1.39753079, -1.84387477],
       [ 4.        ,  2.        ,  1.        , ...,  1.        ,
        -1.3752838 , -1.84387477],
       [ 0.        ,  2.        ,  1.        , ...,  1.        ,
        -1.39753079, -1.84387477],
       ...,
       [ 5.        ,  1.        ,  1.        , ...,  0.        ,
         0.22371837,  1.69569214],
       [ 5.        ,  1.        ,  1.        , ...,  0.        ,
        -0.30881888,  1.69569214],
       [ 5.        ,  1.        ,  4.        , ...,  0.        ,
        -0.29769538,  1.69569214]])

In [18]:
y

0          5953
1          5953
2          5956
3          5955
4          5955
          ...  
300148    69265
300149    77105
300150    79099
300151    81585
300152    81585
Name: price, Length: 300153, dtype: int64

In [19]:

# separation dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((240122, 9), (60031, 9))

In [190]:
#from sklearn.decomposition import PCA

In [17]:
#pca = PCA()
#pca.fit(X_train)
#cumsum = np.cumsum(pca.explained_variance_ratio_)
#d = np.argmax(cumsum >= 0.95) + 1

In [19]:
#pca = PCA(n_components=0.95)
#X_reduced = pca.fit_transform(X_train)

#### Sprawdzenie przeuczenia modelu

In [20]:
model = Ridge()
model.fit(X_train, y_train)

Ridge()

In [21]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

In [22]:
test_score


0.9045543678932391

In [23]:
train_score

0.9046217086611048

In [24]:
if train_score > test_score:
    print("Model jest przeuczony")
else:
    print("Model jest niedouczony")

Model jest przeuczony


#### Cross validation check below

In [30]:
scores = cross_val_score(model, X, y, cv=5)

print("Cross-validation:", scores)

mean_score = scores.mean()
std_score = scores.std()

print("Average Score:", mean_score)
print("Standard deviation:", std_score)

Cross-validation: [-0.20699547 -0.20364342 -0.15842828  0.83777997  0.07666835]
Average Score: 0.06907623152688783
Standard deviation: 0.39832473456929113


#### Create an Evaluate Function to give all metrics after model Training

In [256]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    model.score(true, predicted)
   
    
    
    return mae, rmse, r2_square

In [257]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
  

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 7009.3228
- Mean Absolute Error: 4646.8142
- R2 Score: 0.9046
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7014.3097
- Mean Absolute Error: 4624.9949
- R2 Score: 0.9046


Lasso
Model performance for Training set
- Root Mean Squared Error: 7009.3234
- Mean Absolute Error: 4646.5568
- R2 Score: 0.9046
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7014.3014
- Mean Absolute Error: 4624.7218
- R2 Score: 0.9046


Ridge
Model performance for Training set
- Root Mean Squared Error: 7009.3228
- Mean Absolute Error: 4646.8572
- R2 Score: 0.9046
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7014.3090
- Mean Absolute Error: 4625.0357
- R2 Score: 0.9046


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 3078.8466
- Mean Absolute Error: 1439.7873


In [258]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
5,Random Forest Regressor,0.984990
4,Decision Tree,0.975472
6,XGBRegressor,0.975145
7,CatBoosting Regressor,0.974130
3,K-Neighbors Regressor,0.970030
8,AdaBoost Regressor,0.914911
1,Lasso,0.904555
2,Ridge,0.904554
0,Linear Regression,0.904554
